In [ ]:
# !pip install -q git+https://github.com/seankmartin/simuran.git@main
# !pip install -q allensdk==2.14.1
# !pip install -q one-api==1.17.0
# !pip install -q ibllib==2.19.0

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
import simuran as smr
from simuran.loaders.allen_loader import AllenVisualBehaviorLoader
from simuran.loaders.one_loader import OneAlyxLoader
import pandas as pd
from simuran.bridges.allen_vbn_bridge import (
    filter_good_units,
    get_brain_regions_to_structure_dict,
)
from simuran.plot.unit import plot_unit_properties

In [ ]:
try:
    from google.colab import drive
    drive.mount("/content/drive")
    data_directory = Path("/content/drive/MyDrive/NeuroScience/OpenData/")
except ModuleNotFoundError:
    data_directory = Path(r"G:\\OpenData")

In [ ]:
allen_name = "allen_npixels2"
ibl_name = "IBLdata"
ALLEN_MANIFEST = "visual-behavior-neuropixels_project_manifest_v0.4.0.json"
output_directory = data_directory / "OpenDataResults"
allen_cache_dir = data_directory / allen_name
ibl_cache_dir = data_directory / ibl_name

In [ ]:
# allen_loader = AllenVisualBehaviorLoader(
#     cache_directory=allen_cache_dir, manifest=ALLEN_MANIFEST
# )
# allen_sessions = allen_loader.get_sessions_table()
one_loader = OneAlyxLoader(cache_directory=ibl_cache_dir)
one_sessions = one_loader.get_sessions_table()

allen_recording_container = smr.RecordingContainer.from_table(allen_sessions, allen_loader)
one_recording_container = smr.RecordingContainer.from_table(one_sessions, one_loader)

In [ ]:
example_one = one_recording_container.load(0, force_reload=True)

In [ ]:
# from simuran.bridges.allen_vbn_bridge import allen_to_spike_train
# example_recording = allen_recording_container.load(0)
# units_df, spike_train = allen_to_spike_train(example_recording)

In [ ]:
from simuran.bridges.ibl_wide_bridge import one_spike_train, one_trial_info

unit_table, spike_train = one_spike_train(example_one)
trial_info = one_trial_info(example_one)

In [ ]:
def split_spikes_into_trials(spike_train, trial_info, num_trials=None):
    new_spike_train = []
    for i, (start, end) in enumerate(trial_info["trial_times"]):
        if (num_trials is not None) and (i == num_trials):
            break
        for k, v in spike_train.items():
            to_use = v[(v >= start) & (v < end)]
            new_spike_train.append(to_use - start)
    return new_spike_train

In [ ]:
small_train = smaller_spike_train(spike_train, 20, 10)

In [ ]:
from simuran.bridges.neo_bridge import convert_spikes_to_train
neo_train = convert_spikes_to_train(small_train)

In [ ]:
from elephant.gpfa import GPFA
import quantities as pq

bin_size = 20 * pq.ms 

gpfa_3dim = GPFA(x_dim=3, bin_size=bin_size)

# Axis 0 is trials
# Axis 1 is neurons
# Axis 2 is the times
gpfa_3dim.fit([neo_train])

In [ ]:
trajectories = gpfa_3dim.transform([neo_train])

In [ ]:
trajectories

In [ ]:
import numpy as np
average_trajectory = np.mean(trajectories, axis=0)
average_trajectory

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=plt.figaspect(1.0))
ax = fig.add_subplot(projection="3d")
ax.plot(*average_trajectory)
plt.show()
plt.close(fig)